In [1]:
%pip install groq python-dotenv --upgrade --quiet

import os
from groq import Groq
from dotenv import load_dotenv

load_dotenv()

# Set up your Groq API Key
# If not in your .env file, you'll be prompted to enter it
if "GROQ_API_KEY" not in os.environ:
    import getpass
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.9 MB/s eta 0:00:00
Enter your Groq API Key: ··········


In [9]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a Senior Technical Support Engineer at RocketBoots. Provide rigorous, code-focused, and precise solutions. If code is involved, provide a clean snippet.",
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": "You are a Billing Specialist at RocketBoots. Be empathetic and professional. Focus on financial accuracy and company refund policies.",
        "temperature": 0.7
    },
    "general": {
        "system_prompt": "You are a friend. Have a casual conversation with a warm,friendly and empathetic tone.",
        "temperature": 0.7
    }
}

In [15]:
def route_prompt(user_input):
    # This instruction is designed to satisfy the "only category name" constraint
    router_instruction = f"Classify this text into one of these categories: [technical, billing, general]. Return ONLY the word. Query: {user_input}"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": router_instruction}],
        temperature=0 # Consistency is key for routing
    )

    # Return ONLY the category name as a cleaned string
    return response.choices[0].message.content.strip().lower()

# Verification: This will show you the output of just the router
print(f"Router Test Output: {route_prompt('My python script is throwing an IndexError on line 5.')}")

Router Test Output: technical


In [16]:
def process_request(user_input):
    # 1. Calls route_prompt to decide the category
    category = route_prompt(user_input)

    # Fallback to general if the router returns something unexpected
    if category not in MODEL_CONFIG:
        category = "general"

    print(f"--- Routing to: {category.upper()} EXPERT ---")

    # 2. Selects the correct System Prompt based on the category
    config = MODEL_CONFIG[category]
    expert_prompt = config["system_prompt"]

    # 3. Calls the generic LLM with specific System Prompt + User Input
    # Using Llama 3.1 8B as our base expert model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": expert_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=config["temperature"] # Experts get higher temp for natural answers
    )

    # 4. Returns the final answer
    return response.choices[0].message.content

# --- EXECUTION ---
# This is where we call the orchestrator to see the final result
query = "I was charged twice for my subscription this month."
print("\nFINAL ANSWER:")
print(process_request(query))


FINAL ANSWER:
--- Routing to: BILLING EXPERT ---
I'm so sorry to hear that you've been charged twice for your subscription this month. I can imagine how frustrating that must be for you.

Firstly, please know that we take situations like this very seriously, and I'm here to help you resolve the issue as quickly as possible.

To assist you further, can you please provide me with your subscription details, including your email address associated with your RocketBoots account, as well as the dates of the duplicate charges? This will enable me to investigate the matter and identify the cause of the error.

Once I've located the issue, I'll work with our team to correct the mistake and process a refund for the duplicate charge. I'll also ensure that our system is updated to prevent such errors from happening in the future.

In the meantime, if you're experiencing any financial hardship due to the duplicate charge, please let me know, and I'll do my best to provide you with temporary assist